In [1]:
import pandas as pd
import math

In [2]:
# WIN
users_sleepy = pd.read_csv(r'C:\Users\Incognitus\Downloads\entries.csv', sep=';') 
# MAC
#users_sleepy = pd.read_csv(r'/Users/vladislavlipkin/Downloads/entries.csv', sep=';') 

In [8]:
# ===============================================================
# 1️⃣ ПОДГОТОВКА ДАННЫХ
# ===============================================================

# Берем нужные столбцы из исходного датафрейма (id пользователя и дата активности)
users_filter = users_sleepy.loc[:, ['user', 'dt']]

# Преобразуем дату в формат datetime, чтобы с ней можно было работать как с датой
users_filter['dt'] = pd.to_datetime(users_filter['dt'], format='%d.%m.%Y')

# Определяем дату первого визита (регистрации) каждого пользователя
# — transform('min') вернет для каждого пользователя дату его первого визита
users_filter['dt_reg'] = users_filter.groupby('user')['dt'].transform('min')

# Считаем количество дней, прошедших с даты регистрации
# — это поможет определить "n-day retention"
users_filter['diff'] = (users_filter['dt'] - users_filter['dt_reg']).dt.days

# Добавляем количество недель (целое округление)
# — n-week retention будет рассчитываться на основе этого признака
users_filter['week'] = round(users_filter['diff'] / 7, 0)

# Добавляем количество месяцев (приблизительно считаем 30 дней за месяц)
users_filter['month'] = round(users_filter['diff'] / 30, 0)


# ===============================================================
# 2️⃣ РАСЧЕТ RETENTION (по дням, неделям, месяцам)
# ===============================================================

# Считаем общее количество уникальных пользователей — это наша база для процента
all_users = users_filter['user'].nunique()

# Вычисляем retention по дням (n-day): доля уникальных пользователей в каждом diff
users_filter['N_DAY_retention'] = (
    users_filter.groupby('diff')['user'].transform('nunique') / all_users * 100
)

# Вычисляем retention по неделям (n-week)
users_filter['N_WEEK_retention'] = (
    users_filter.groupby('week')['user'].transform('nunique') / all_users * 100
)

# Вычисляем retention по месяцам (n-month)
users_filter['N_MONTH_retention'] = (
    users_filter.groupby('month')['user'].transform('nunique') / all_users * 100
)


# ===============================================================
# 3️⃣ УНИВЕРСАЛЬНОЕ УПРАВЛЕНИЕ ЧЕРЕЗ ОДНУ ПЕРЕМЕННУЮ
# ===============================================================

# Меняешь только эту строку — и код перестраивается под нужный масштаб времени
# Возможные значения: 'day', 'week', 'month'
period = 'week'


# Определяем, по какому признаку группировать и как называть в выводе
if period == 'day':
    idx_col = 'diff'
    period_name = 'Day'
elif period == 'week':
    idx_col = 'week'
    period_name = 'Week'
elif period == 'month':
    idx_col = 'month'
    period_name = 'Month'
else:
    raise ValueError("period must be 'day', 'week', or 'month'")


# ===============================================================
# 4️⃣ СОЗДАНИЕ СВОДНОЙ ТАБЛИЦЫ RETENTION
# ===============================================================

# Считаем количество уникальных пользователей для каждого дня/недели/месяца
pivot = (
    users_filter
    .groupby(idx_col, as_index=False)['user']
    .nunique()
    .rename(columns={'user': 'unique_users'})
)

# Добавляем процент удержания (retention)
pivot['retention'] = pivot['unique_users'] / all_users * 100

# Добавляем название периода для удобства анализа/экспорта
pivot['period'] = period_name


# ===============================================================
# 5️⃣ ДОБАВЛЕНИЕ ПОЛНОГО ДИАПАЗОНА (чтобы не было пропусков)
# ===============================================================

# Создаем диапазон от минимального до максимального значения периода
# — полезно, если retention не было рассчитано для некоторых дней/недель
full_range = pd.DataFrame({
    idx_col: range(int(pivot[idx_col].min()), int(pivot[idx_col].max()) + 1)
})

# Объединяем полный диапазон с рассчитанными значениями retention
pivot_full = full_range.merge(pivot, on=idx_col, how='left')


# ===============================================================
# 6️⃣ ВЫВОД РЕЗУЛЬТАТА
# ===============================================================

print(f"Retention by {period_name}")
display(pivot_full.head(10))  # можно заменить на pivot_full для полного вывода

print(f'LTV: {pivot_full['retention'].sum() / 100} , methood by integral')

Retention by Week


,week,unique_users,retention,period
0,0,2000,100.00,Week
1,1,1812,90.60,Week
2,2,1660,83.00,Week
3,3,1476,73.80,Week
4,4,1395,69.75,Week
5,5,1344,67.20,Week
6,6,1339,66.95,Week
7,7,1264,63.20,Week
8,8,1205,60.25,Week
9,9,1143,57.15,Week


LTV: 14.5975 , methood by integral
